# Linear Regression 

Finite Differences method

In [2]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import random
import matplotlib.colors as mcolors
from ipywidgets import interact, widgets, interactive, fixed, interact_manual


import pickle

with open("Teach_linear_regression.pkl", "rb") as f:
    (
        x, y, COEF_A, COEF_B,
        COEF_A_MIN, COEF_A_MAX,
        COEF_B_MIN, COEF_B_MAX,
        WORST_A_PARAM, WORST_B_PARAM,
        MIN_COST, MAX_COST,
        MIN_VALUE, MAX_VALUE, NB_ELEMENTS,
    ) = pickle.load(f)

In [45]:
%matplotlib inline

def predict(x, thetas):
    a, b = thetas
    y_hat = a * x + b
    return y_hat


def get_cost(y, y_hat):
    return ((y - y_hat) ** 2).sum()


def get_finite_differences(x, y, thetas) -> np.array:
    epsilon = 1e-1

    y_hat = predict(x, thetas)
    cost = get_cost(y, y_hat)

    y_hat_a_epsilon = predict(x, thetas + np.array([epsilon, 0]))
    cost_a_epsilon = get_cost(y, y_hat_a_epsilon)

    y_hat_b_epsilon = predict(x, thetas + np.array([0, epsilon]))
    cost_b_epsilon = get_cost(y, y_hat_b_epsilon)

    return np.array([cost_a_epsilon - cost, cost_b_epsilon - cost]), cost

def do_finite_differences(steps, thetas, learning_rate=1e-1):
    thetas_history = []
    cost_history = []
    finite_differences_history = []
    for _ in range(steps):
        last_finite_differences, cost = get_finite_differences(x, y, thetas)
        thetas = thetas - learning_rate * last_finite_differences
        thetas_history.append(thetas)
        cost_history.append(cost)
        finite_differences_history.append(last_finite_differences)
    thetas_history = np.array(thetas_history)
    cost_history = np.array(cost_history)
    finite_differences_history = np.array(finite_differences_history)
    return thetas, thetas_history, cost_history, finite_differences_history


def get_cost_for_all_thetas():
    COEF_A_MIN, COEF_A_MAX
    COEF_B_MIN, COEF_B_MAX
    COEF_A_NB_POINTS = 100
    COEF_B_NB_POINTS = 100
    value_of_a_and_cost = []
    value_of_b_and_cost = []
    for a in np.linspace(COEF_A_MIN, COEF_A_MAX, COEF_A_NB_POINTS):
        b = COEF_B
        thetas = np.array([a, b])
        cost = get_cost(y, predict(x, thetas))
        value_of_a_and_cost.append((a, cost))
    for b in np.linspace(COEF_B_MIN, COEF_B_MAX, COEF_B_NB_POINTS):
        a = COEF_A
        thetas = np.array([a, b])
        cost = get_cost(y, predict(x, thetas))
        value_of_b_and_cost.append((b, cost))
    return np.array(value_of_a_and_cost), np.array(value_of_b_and_cost)


def plot_finite_differences(thetas_history, cost_history, finite_differences_history, plot_cost_surface=False):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Convert to numpy arrays for easier indexing
    thetas_history = np.array(thetas_history)
    cost_history = np.array(cost_history)
    
    # Create color gradient from red to green
    colors = np.zeros((len(thetas_history), 3))
    colors[:, 0] = np.linspace(1, 0, len(thetas_history))  # Red channel
    colors[:, 1] = np.linspace(0, 1, len(thetas_history))  # Green channel
    
    # First plot: Cost vs Theta_0 (a parameter)
    scatter1 = ax1.scatter(thetas_history[:, 0], cost_history, c=colors)
    ax1.set_xlabel('Theta 0 (a)')
    ax1.set_ylabel('Cost')
    
    delta = .2
    # Add slope line for last point of theta_0
    last_x0 = thetas_history[-1, 0]
    last_y0 = cost_history[-1]
    slope0 = finite_differences_history[-1, 0]
    x0_line = np.array([last_x0 - delta, last_x0 + delta])
    y0_line = slope0 * (x0_line - last_x0) + last_y0
    ax1.plot(x0_line, y0_line, 'r--', alpha=0.5, linewidth=5, linestyle='--', color='blue')
    
    # Second plot: Cost vs Theta_1 (b parameter)
    scatter2 = ax2.scatter(thetas_history[:, 1], cost_history, c=colors)
    ax2.set_xlabel('Theta 1 (b)')
    ax2.set_ylabel('Cost')
    
    # Add slope line for last point of theta_1
    last_x1 = thetas_history[-1, 1]
    last_y1 = cost_history[-1]
    slope1 = finite_differences_history[-1, 1]
    x1_line = np.array([last_x1 - delta, last_x1 + delta])
    y1_line = slope1 * (x1_line - last_x1) + last_y1
    ax2.plot(x1_line, y1_line, 'r--', alpha=0.5, linewidth=5, linestyle='--', color='blue')
    
    if plot_cost_surface:
        value_of_a_and_cost, value_of_b_and_cost = get_cost_for_all_thetas()
        print(f"{value_of_a_and_cost.shape = }")
        print(f"{value_of_b_and_cost.shape = }")
        ax1.plot(value_of_a_and_cost[:, 0], value_of_a_and_cost[:, 1], color='gray', linestyle=':')
        ax2.plot(value_of_b_and_cost[:, 0], value_of_b_and_cost[:, 1], color='gray', linestyle=':')

    plt.tight_layout()




# Mieux expliquer : quon veut observer le chamngement de ocut en modifiant un peu nos parametres

In [46]:
@interact(
    steps=widgets.IntSlider(
        min=1,
        max=50,
        value=1,
        # value=(COEF_A_MIN + COEF_A_MAX) / 2,
    ),
    a=widgets.FloatSlider(
        min=COEF_A_MIN,
        max=COEF_A_MAX,
        value=WORST_A_PARAM,
        # value=(COEF_A_MIN + COEF_A_MAX) / 2,
    ),
    b=widgets.FloatSlider(
        min=COEF_B_MIN,
        max=COEF_B_MAX,
        value=WORST_B_PARAM,
        # value=(COEF_B_MIN + COEF_B_MAX) / 2
    ),
    learning_rate=widgets.FloatSlider(
        min=1e-3,
        max=1e-1,
        value=1e-2,
    ),
    show_loss_curve=widgets.Checkbox(value=False, description="Show loss curve"),
)
def interactive_finite_differences(steps, a, b, learning_rate, show_loss_curve):
    thetas = np.array([a, b])
    print(f"{thetas = }")
    thetas, thetas_history, cost_history, finite_differences = do_finite_differences(
        steps, thetas, learning_rate=learning_rate
    )
    print(f"{thetas_history.shape = }")
    print(f"{cost_history.shape = }")
    print(f"{finite_differences.shape = }")
    plot_finite_differences(
        thetas_history,
        cost_history,
        finite_differences,
        plot_cost_surface=show_loss_curve,
    )

interactive(children=(IntSlider(value=1, description='steps', max=50, min=1), FloatSlider(value=0.0, descripti…